In [1]:
!pip install qiskit

      Successfully uninstalled sympy-1.1.1
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import matplotlib.pyplot as plt
import numpy as np

from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name

In [3]:
seed = 10600
aqua_globals.random_seed = seed

In [4]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [6]:
import io
import pandas as pd
dat = pd.read_csv(io.BytesIO(uploaded['data.csv']))

In [7]:
safe = []
unsafe = []
for i in range(len(dat)):
    if dat['Safe(1)/Unsafe(0)'].values[i] == 1:
        safe.append([dat['Case rate(weekly)'].values[i],dat['Test Rate(weekly)'].values[i]])
    else: 
        unsafe.append([dat['Case rate(weekly)'].values[i],dat['Test Rate(weekly)'].values[i]])

In [9]:
print(len(safe))
print(len(unsafe))

31
24


In [30]:
feature_dim = 2
training_input = {'safe' : np.array(safe[0:-1]), 'unsafe' : np.array(unsafe[0:-1])} 
test_input = {'safe' : np.array(safe[0:16]), 'unsafe' : np.array(unsafe[0:12])} 
class_labels = {'unsafe', 'safe'}

In [31]:
onearr = [1]*len(safe)
zeroarr = [0]*len(unsafe)
label = np.concatenate((onearr,zeroarr))
print(label)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [32]:
join = safe + unsafe
print(join)

[[50, 2294], [48, 3195], [27, 1364], [41, 1801], [66, 3055], [45, 1478], [57, 1321], [44, 1578], [54, 3038], [55, 2474], [59, 2649], [30, 4900], [68, 3946], [32, 5063], [62, 2578], [50, 4506], [24, 1632], [50, 6354], [50, 3216], [53, 2859], [68, 5089], [64, 1754], [32, 8090], [61, 2228], [28, 2420], [36, 1542], [55, 4981], [55, 2334], [63, 2236], [66, 2110], [66, 1963], [83, 4624], [72, 2140], [86, 1802], [114, 2025], [71, 1539], [102, 1695], [74, 2564], [118, 2694], [89, 4118], [76, 1820], [90, 2267], [69, 2794], [71, 2656], [161, 2974], [85, 3629], [71, 2956], [137, 2156], [128, 8184], [85, 1345], [94, 2097], [76, 3190], [73, 1802], [69, 1760], [101, 3687]]


In [33]:
datapoints = [np.array(join),label]
class_to_label = {'unsafe': 0, 'safe': 1}

In [36]:
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2, entanglement='linear')
qsvm = QSVM(feature_map, training_input, test_input, datapoints[0])

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=8192, seed_simulator=seed, seed_transpiler=seed)

result = qsvm.run(quantum_instance)

print(f'Testing success ratio: {result["testing_accuracy"]}')
print()
print('Prediction from datapoints set:')
print(f'  ground truth: {map_label_to_class_name(datapoints[1], qsvm.label_to_class)}')
print(f'  prediction:   {result["predicted_classes"]}')
predicted_labels = result["predicted_labels"]
print(f'  success rate: {100*np.count_nonzero(predicted_labels == datapoints[1])/len(predicted_labels)}%')

Testing success ratio: 0.39285714285714285

Prediction from datapoints set:
  ground truth: ['unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe', 'safe']
  prediction:   ['unsafe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'unsafe', 'safe', 'unsafe', 'safe', 'safe', 'safe', 'unsafe', 'safe', 'unsafe', 'safe', 'safe', 'safe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'safe', 'safe', 'safe', 'safe', 'unsafe', 'unsafe', 'safe', 'safe', 'safe', 'unsafe', 'safe', 'safe', 'safe', 'uns